In [2]:
import os
import json
import google.cloud.bigquery as bq
import pandas as pd

In [6]:
%load_ext google.cloud.bigquery

In [7]:
%%bigquery station_sample

SELECT
  *
FROM `bigquery-public-data.ghcn_d.ghcnd_2000` ghcnd_2000
WHERE
  ghcnd_2000.id = 'GPM00078894'
  AND ghcnd_2000.element ='TAVG'
LIMIT 1000

In [17]:
station_sample.sort_values(['date', 'time']).head(10)

,id,date,element,value,mflag,qflag,sflag,time
188,GPM00078894,2000-01-28,TAVG,249.0,H,None,S,None
172,GPM00078894,2000-01-29,TAVG,244.0,H,None,S,None
274,GPM00078894,2000-01-30,TAVG,237.0,H,None,S,None
261,GPM00078894,2000-01-31,TAVG,241.0,H,None,S,None
233,GPM00078894,2000-02-01,TAVG,241.0,H,None,S,None
62,GPM00078894,2000-02-02,TAVG,247.0,H,None,S,None
293,GPM00078894,2000-02-03,TAVG,247.0,H,None,S,None
327,GPM00078894,2000-02-04,TAVG,251.0,H,None,S,None
73,GPM00078894,2000-02-05,TAVG,256.0,H,None,S,None
56,GPM00078894,2000-02-06,TAVG,251.0,H,None,S,None


In [13]:
%%bigquery hrcn_sample

SELECT
  sid,
  iso_time,
  latitude,
  longitude,
  WMO_WIND
FROM
  `bigquery-public-data.noaa_hurricanes.hurricanes`
WHERE
  extract(year FROM iso_time) = 2000
  AND sid = "1999357S08076"
LIMIT 1000

In [16]:
hrcn_sample.sort_values('iso_time').head(10)

,sid,iso_time,latitude,longitude,WMO_WIND
5,1999357S08076,2000-01-01 00:00:00+00:00,-12.7667,49.0000,25.0
13,1999357S08076,2000-01-01 03:00:00+00:00,-12.7476,48.4301,NaN
6,1999357S08076,2000-01-01 06:00:00+00:00,-12.7333,47.9000,25.0
9,1999357S08076,2000-01-01 09:00:00+00:00,-12.6807,47.5150,NaN
8,1999357S08076,2000-01-01 12:00:00+00:00,-12.6333,47.2000,25.0
10,1999357S08076,2000-01-01 15:00:00+00:00,-12.6183,46.8858,NaN
3,1999357S08076,2000-01-01 18:00:00+00:00,-12.6333,46.5667,30.0
16,1999357S08076,2000-01-01 21:00:00+00:00,-12.6708,46.2150,NaN
1,1999357S08076,2000-01-02 00:00:00+00:00,-12.7333,45.8333,33.0
14,1999357S08076,2000-01-02 03:00:00+00:00,-12.8091,45.4366,NaN


In the `hurricanes` sample, there are several measurements per day, this could potentially also apply to the `temperatures` sample as the `time` column indicates.

In [19]:
hrcn_sample.set_index('iso_time').resample('1D').mean()

,latitude,longitude,WMO_WIND
iso_time,,,
2000-01-01 00:00:00+00:00,-12.685500,47.464075,26.250000
2000-01-02 00:00:00+00:00,-13.059137,44.062050,34.500000
2000-01-03 00:00:00+00:00,-13.480500,40.345160,29.333333


We can add a temporal aggregation to the SQL queries and join the results.

In [27]:
%%bigquery hrcn_agg_sample

SELECT
  sid,
  DATE(iso_time) AS date,
  AVG(latitude)  AS lat,
  AVG(longitude) AS lng,
  AVG(WMO_WIND)  AS mean_wind_speed
FROM
  `bigquery-public-data.noaa_hurricanes.hurricanes`
WHERE
  EXTRACT(year FROM iso_time) = 2000
  AND sid = "1999357S08076"
GROUP BY
  sid, date
LIMIT 1000

In [29]:
hrcn_agg_sample.sort_values('date')

,sid,date,lat,lng,mean_wind_speed
1,1999357S08076,2000-01-01,-12.685500,47.464075,26.250000
0,1999357S08076,2000-01-02,-13.059138,44.062050,34.500000
2,1999357S08076,2000-01-03,-13.480500,40.345160,29.333333


In [31]:
%%bigquery station_agg_sample

SELECT
  DATE(date) AS date,
  AVG(value) AS avg_temp
FROM `bigquery-public-data.ghcn_d.ghcnd_2000` ghcnd_2000
WHERE
  ghcnd_2000.id = 'GPM00078894'
  AND ghcnd_2000.element ='TAVG'
GROUP BY
 date
LIMIT 1000

In [35]:
station_agg_sample.sort_values('date').head()

,date,avg_temp
300,2000-01-28,249.0
152,2000-01-29,244.0
196,2000-01-30,237.0
191,2000-01-31,241.0
161,2000-02-01,241.0
